# COMP30024 Artificial Intelligence Part A

In this first part of the project, you will solve a simple search-based problem on the Cachex game board.
Before you read this specification, please make sure you have carefully read the entire ‘Rules for the Game
of Cachex’ document. Although you won’t be writing an agent to play the game just yet, you should be
aiming to get very familiar with the board layout and corresponding hex coordinate system.

The aims for Project Part A are for you and your project partner to
- refresh and extend your Python programming skills
- explore some of the algorithms you have encountered in lectures, and 
- become more familiar with the Cachex task environment.

This is also a chance for you to develop fundamental Python tools for working with the game: Some of the functions and classes you create now may be helpful later (when you are building your full game-playing program for Part B of the project).

To compute this path you are asked to use an A* search and design an **admissible heuristic** to optimise its performance. There are a number of assumptions you should make:
1. The start and goal cells will always be unoccupied (but any other cells may be occupied).
2. All given cell coordinates will be within the bounds of a board of size n. More precisely, for any given cell coordinate (r, q), 0 ≤ r < n and 0 ≤ q < n. You may also assume n ≥ 1.
3. The cost of a path is defined as the number of cells that form a continuous path from the start cell to the goal cell (including these cells).
4. If there is a tie, that is, multiple minimal paths of the same cost exist on the same board configuration, any such path is a valid solution.
5. There may not always be a valid path from the 

In [25]:
# import required libraries
import json
from math import pow, sqrt, fabs

In [8]:
# obtain board data via open the sample_input.json
# this should be read via system args
with open('sample_input.json') as json_file:
    # read cachex board game data as a const variable
    BOARD_DATA = json.load(json_file)

In [9]:
BOARD_DATA

{'n': 5,
 'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]],
 'start': [4, 2],
 'goal': [0, 0]}

In [55]:
def construct_board(n: int):
    """
    The function will return all valid hexagon cell coordinates in a single
    set
    input: n: int # number of the board size
    return: 
    """
    board = set()

    # construct cachex board
    for r in range(BOARD_DATA['n']):
        for q in range(BOARD_DATA['n']):
            board.add((r, q))
    return board

In [87]:
# define the error type
class Error(Exception):
    """
    Cachex AStar Path Solver Error
    """
    pass

class InvalidHeuristicError(Error):
    """
    Heuristic function must be one of the following distance formula:
    ['euclidean', 'manhatten', 'hamming']
    """
    def __init__(self):
        self.message = "Heuristic function must be one of the following distance formula: ['euclidean', 'manhatten', 'hamming']"
        super().__init__(self.message)
        
class InvalidNodeStateError(Error):
    """
    Node only have four possible state status:
    ['Red', 'Blue', 'Block', None]
    """
    
    def __init__(self):
        self.message = "Node only have four possible state status: ['Red', 'Blue', 'Block', None]"
        super().__init__(self.message)

In [88]:
board = construct_board(BOARD_DATA['n'])

In [129]:
# define the node class
class HexNode:
    """
    In Cachex game, each Hexagon cell will be represented with a object HexNode,
    where HexNode contains its coordinates, next valid moves, current hexagon cell 
    status.
    input: coords: tuple, move: list, state: string or None
    return: class HexNode
    """
    def __init__(self, coords: tuple, state=None):
        self.coord = coords
        self.next = set()
        
        if state not in ['Red', 'Blue', 'Block', None]:
            raise InvalidNodeStateError
        self.state = None # state could be Red, Blue, Block or None
        
    def distance_diff(self, target, heuristic='manhatten', p=None):
        """
        Calculate the distance between current node and the target hexagon cell using
        given heuristic distance function
        
        heuristic must be one of the following distance formula:
        ['euclidean', 'manhatten', 'hamming']
        """
        if heuristic not in ['euclidean', 'manhatten', 'minkowski']:
            raise InvalidHeuristicError
        
        # calculate the distance with the given heuristic distance formula
        if heuristic is 'euclidean':
            return self.minkowski(self.coord, target, 2)
        elif heuristic is 'manhatten':
            return self.minkowski(self.coord, target, 1)
        elif heuristic is 'minkowski':
            return self.minkowski(self.coord, target, p)
    
    def minkowski(self, point1: tuple, point2: tuple, p:int):
        """
        Calculate the distance use minkowski distance formula where
        distance = (sum( abs(point1[0] - point2[0])^p, abs(point1[1] - point2[1])^p ))**(1/p)
        
        where p = 1, minkowski == manhatten distance
        where p = 2, minkowski == euclidean distance
        """
        return pow(pow(abs(point1[0] - point2[0]), p) + pow(abs(point1[1]-point2[1]), p), 1/p)
    
    def find_next_moves(self, board, inplace=True):
        """
        Through obversation, if turn the Cachex game board from a hexagon 2D layout to a rectangle grid
        layout, a node could move in six directions except a node cannot move along the major axis.
        
        Hence for each point:
        1. check all posible moves
        2. check moves are vaild (in board)
        3. return a result list
        
        board: set # game board coordinates information to check a move is valid or not
        inplace: boolean # could return a set instead changing node attribute value
        """
        
        # define the output result
        possible_moves = set()
        
        # generate possible moves
        for r in range(self.coord[0]-1, self.coord[0]+2):
            for q in range(self.coord[1],  self.coord[0]+2):
                if (r, q) in board:
                    possible_moves.add((r, q))
        
        print(possible_moves)
        # remove the diagonal elements along the major axis
        if (self.coord[0]-1, self.coord[1]-1) in possible_moves:
            possible_moves.remove((self.coord[0]-1, self.coord[1]-1))
        if (self.coord[0]+1, self.coord[1]+1) in possible_moves:
            possible_moves.remove((self.coord[0]+1, self.coord[1]+1))
        
        # return result
        if inplace is True:
            self.next = possible_moves
            return
        return possible_moves

In [130]:
node = HexNode(coords=(1, 1), state=None)

In [131]:
node.find_next_moves(board=board)

{(0, 1), (1, 2), (0, 2), (2, 1), (2, 2), (1, 1)}
